# ETL: Silver → Gold

## Contexto do Projeto

Este notebook realiza o processo de ETL (Extract, Transform, Load) para transformar dados da camada Silver para a camada Gold em um pipeline de dados. A camada Gold é destinada a análises avançadas e relatórios, contendo dados limpos, agregados e otimizados para consultas.

## Processo ETL
1. **Extração (Extract)**:
   - Os dados são extraídos da tabela Silver, que contém dados semi-estruturados e pré-processados. 

2. **Transformação (Transform)**:
    - Cria dimensões e fatos a partir dos dados extraídos (Star Schema).

3. **Carga (Load)**:
   - Os dados transformados são carregados no schema Gold.

## 1. Configuração e Imports

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Configuração de conexão com o banco de dados PostgreSQL
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5433'),
    'database': os.getenv('DB_NAME', 'grupo08'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

## 2. Extração de Dados da Camada Silver

In [3]:
# Estabelecendo a conexão com o banco de dados
try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("Conexão com o banco de dados estabelecida com sucesso.")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")

# Extração de Dados da Camada Silver
silver_table = "silver.filmes"
try:
    df = pd.read_sql_query(f"SELECT * FROM {silver_table};", conn)
    print("Dados extraídos com sucesso.")
    print(f"Número de registros extraídos: {len(df)}")
    print(f"Número de colunas extraídas: {len(df.columns)}")
except Exception as e:
    print(f"Erro ao extrair dados: {e}")

Conexão com o banco de dados estabelecida com sucesso.
Dados extraídos com sucesso.
Número de registros extraídos: 1111347
Número de colunas extraídas: 29


## 3. Transformação dos Dados e Carga na Camada Gold

Nesta etapa, será criado o Star Schema contendo as tabelas de dimensão e fato. Utilizaremos Surrogate Keys (SK) e aplicaremos o padrão mnemônico na nomenclatura das colunas.

In [4]:
# Criando schema Gold se não existir
ddl_schema = "CREATE SCHEMA IF NOT EXISTS gold;"

try:
    with conn.cursor() as cursor:
        cursor.execute(ddl_schema)
        conn.commit()
    print("Schema 'gold' verificado/criado com sucesso.")
except Exception as e:
    print(f"Erro ao criar/verificar schema 'gold': {e}")

Schema 'gold' verificado/criado com sucesso.


In [5]:
print ("Criando as tabelas fato e dimensão na camada Gold...")

# 1. DIMENSÃO RELEASE (CALENDÁRIO)
try:
    with conn.cursor() as cur:
        print("Criando dimensão Release...")
        
        sql_dim_release = """
        DROP TABLE IF EXISTS gold.dim_rel CASCADE;

        CREATE TABLE gold.dim_rel AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY release_date) AS srk_rel,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            release_date          AS rel_dat,
            release_year          AS rel_yer,
            release_month         AS rel_mon,
            release_month_name    AS rel_mon_nam,
            release_day_of_week   AS rel_day_wek,
            release_day_name      AS rel_day_nam,
            release_decade        AS rel_dec
            
        FROM (
            -- Calcula campos temporais a partir de release_date
            SELECT DISTINCT
                release_date,
                EXTRACT(YEAR FROM release_date)::INTEGER AS release_year,
                EXTRACT(MONTH FROM release_date)::INTEGER AS release_month,
                TO_CHAR(release_date, 'Month') AS release_month_name,
                EXTRACT(DOW FROM release_date)::INTEGER AS release_day_of_week,
                TO_CHAR(release_date, 'Day') AS release_day_name,
                (EXTRACT(YEAR FROM release_date)::INTEGER / 10 * 10)::INTEGER AS release_decade
            FROM silver.filmes 
            WHERE release_date IS NOT NULL
        ) sub;

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_rel ADD PRIMARY KEY (srk_rel);
        """
        cur.execute(sql_dim_release)
        print("gold.dim_rel criada com sucesso.")

        df_dim_release = pd.read_sql_query("SELECT * FROM gold.dim_rel;", conn)
        print(f"Número de registros em gold.dim_rel: {len(df_dim_release)}")
        print(f"Número de colunas em gold.dim_rel: {len(df_dim_release.columns)}")
        print(df_dim_release.head())
    
except Exception as e:
    print(f"Erro ao criar gold.dim_rel: {e}")

finally:
    conn.commit()

Criando as tabelas fato e dimensão na camada Gold...
Criando dimensão Release...
gold.dim_rel criada com sucesso.
Número de registros em gold.dim_rel: 37391
Número de colunas em gold.dim_rel: 8
   srk_rel     rel_dat  rel_yer  rel_mon rel_mon_nam  rel_day_wek rel_day_nam  \
0        1  1920-01-01     1920        1   January              4   Thursday    
1        2  1920-01-02     1920        1   January              5   Friday      
2        3  1920-01-03     1920        1   January              6   Saturday    
3        4  1920-01-04     1920        1   January              0   Sunday      
4        5  1920-01-05     1920        1   January              1   Monday      

   rel_dec  
0     1920  
1     1920  
2     1920  
3     1920  
4     1920  


In [6]:
# 2. DIMENSÃO LUCRO

try:
    with conn.cursor() as cur:
        print("Criando dimensão Profit...")
        
        sql_dim_profit = """
        DROP TABLE IF EXISTS gold.dim_pft CASCADE;

        CREATE TABLE gold.dim_pft AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY budget, revenue) AS srk_pft,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            budget          AS bdg,
            revenue         AS rev,
            profit          AS pft,
            roi             AS ret_inv,
            is_profitable   AS pfe,
            budget_tier     AS bdg_tir

        FROM (
            -- Seleciona combinações únicas de valores financeiros
            SELECT DISTINCT 
                budget, revenue, profit, roi, is_profitable, budget_tier
            FROM silver.filmes
        ) sub;

        ALTER TABLE gold.dim_pft ADD PRIMARY KEY (srk_pft);
        """
        cur.execute(sql_dim_profit)
        print("gold.dim_pft criada com sucesso.")

        df_dim_profit = pd.read_sql_query("SELECT * FROM gold.dim_pft;", conn)
        print(f"Número de registros em gold.dim_pft: {len(df_dim_profit)}")
        print(f"Número de colunas em gold.dim_pft: {len(df_dim_profit.columns)}")
        print(df_dim_profit.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()


Criando dimensão Profit...
gold.dim_pft criada com sucesso.
Número de registros em gold.dim_pft: 22166
Número de colunas em gold.dim_pft: 7
   srk_pft  bdg  rev  pft  ret_inv    pfe        bdg_tir
0        1  1.0  1.0  0.0      0.0  False  Micro (< $1M)
1        2  1.0  2.0  1.0    100.0   True  Micro (< $1M)
2        3  1.0  3.0  2.0    200.0   True  Micro (< $1M)
3        4  1.0  4.0  3.0    300.0   True  Micro (< $1M)
4        5  1.0  5.0  4.0    400.0   True  Micro (< $1M)


In [7]:
# 3. DIMENSÃO ENGAGEMENT

try:
    with conn.cursor() as cur:
        print("Criando dimensão Engagement...")
        
        sql_dim_engagement = """
        DROP TABLE IF EXISTS gold.dim_eng CASCADE;

        CREATE TABLE gold.dim_eng AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY popularity DESC) AS srk_eng,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            vote_average    AS vot_avg,
            vote_count      AS vot_cnt,
            popularity      AS pop

        FROM (
            -- Seleciona combinações únicas de engajamento
            SELECT DISTINCT 
                vote_average, vote_count, popularity
            FROM silver.filmes
        ) sub;

        ALTER TABLE gold.dim_eng ADD PRIMARY KEY (srk_eng);
        """
        cur.execute(sql_dim_engagement)
        print("gold.dim_eng criada com sucesso.")

        df_dim_engagement = pd.read_sql_query("SELECT * FROM gold.dim_eng;", conn)
        print(f"Número de registros em gold.dim_eng: {len(df_dim_engagement)}")
        print(f"Número de colunas em gold.dim_eng: {len(df_dim_engagement.columns)}")
        print(df_dim_engagement.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Engagement...
gold.dim_eng criada com sucesso.
Número de registros em gold.dim_eng: 177537
Número de colunas em gold.dim_eng: 4
   srk_eng  vot_avg  vot_cnt       pop
0        1     7.14     1023  2994.357
1        2     8.07      702  2680.593
2        3     6.55      365  1692.778
3        4     6.91     2034  1567.273
4        5     6.82      258  1547.220


In [8]:
# 4. DIMENSÃO LANGUAGE

try:
    with conn.cursor() as cur:
        print("Criando dimensão Language...")
        
        sql_dim_language = """
        DROP TABLE IF EXISTS gold.dim_lng CASCADE;

        CREATE TABLE gold.dim_lng AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY original_language) AS srk_lng,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            original_language AS lng

        FROM (
            -- Seleciona línguas únicas
            SELECT DISTINCT original_language
            FROM silver.filmes
            WHERE original_language IS NOT NULL AND original_language <> ''
        ) sub;

        ALTER TABLE gold.dim_lng ADD PRIMARY KEY (srk_lng);
        """
        cur.execute(sql_dim_language)
        print("gold.dim_lng criada com sucesso.")

        df_dim_language = pd.read_sql_query("SELECT * FROM gold.dim_lng;", conn)
        print(f"Número de registros em gold.dim_lng: {len(df_dim_language)}")
        print(f"Número de colunas em gold.dim_lng: {len(df_dim_language.columns)}")
        print(df_dim_language.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Language...
gold.dim_lng criada com sucesso.
Número de registros em gold.dim_lng: 176
Número de colunas em gold.dim_lng: 2
   srk_lng lng
0        1  aa
1        2  ab
2        3  af
3        4  ak
4        5  am


In [9]:
# 5. DIMENSÃO COMPANY

try:
    print ("Criando dimensão Company...")
    with conn.cursor() as cur:
        
        sql_dim_company = """
        DROP TABLE IF EXISTS gold.dim_cmp CASCADE;

        CREATE TABLE gold.dim_cmp AS
        WITH uniao_companies AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(production_companies, ','))) AS nm_company
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_company) AS nm_company
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_company) AS srk_cmp,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_company AS prd_cmp
            
        FROM uniao_companies
        WHERE nm_company IS NOT NULL AND nm_company <> '';

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_cmp ADD PRIMARY KEY (srk_cmp);
        """
        
        cur.execute(sql_dim_company)
        conn.commit()
        print("gold.dim_cmp criada com sucesso.")
        df_dim_company = pd.read_sql_query("SELECT * FROM gold.dim_cmp;", conn)
        print(f"Número de registros em gold.dim_cmp: {len(df_dim_company)}")
        print(f"Número de colunas em gold.dim_cmp: {len(df_dim_company.columns)}")
        print(df_dim_company.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Company...
gold.dim_cmp criada com sucesso.
Número de registros em gold.dim_cmp: 164247
Número de colunas em gold.dim_cmp: 2
   srk_cmp prd_cmp
0        1     두리컴
1        2     딴생각
2        3     배선환
3        4     씨네넷
4        5     씨네힐


In [10]:
# 6. DIMENSÃO COUNTRY

try:
    print ("Criando dimensão Country...")
    with conn.cursor() as cur:
        
        sql_dim_country = """
        DROP TABLE IF EXISTS gold.dim_ctr CASCADE;
        CREATE TABLE gold.dim_ctr AS
        WITH uniao_countries AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(production_countries, ','))) AS nm_country
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_country) AS nm_country
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_country) AS srk_ctr,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_country AS prd_ctr
            
        FROM uniao_countries
        WHERE nm_country IS NOT NULL AND nm_country <> '';

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_ctr ADD PRIMARY KEY (srk_ctr);
        """
        
        cur.execute(sql_dim_country)
        conn.commit()
        print("gold.dim_ctr criada com sucesso.")
        df_dim_country = pd.read_sql_query("SELECT * FROM gold.dim_ctr;", conn)
        print(f"Número de registros em gold.dim_ctr: {len(df_dim_country)}")
        print(f"Número de colunas em gold.dim_ctr: {len(df_dim_country.columns)}")
        print(df_dim_country.head())
except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Country...
gold.dim_ctr criada com sucesso.
Número de registros em gold.dim_ctr: 250
Número de colunas em gold.dim_ctr: 2
   srk_ctr         prd_ctr
0        1     Afghanistan
1        2         Albania
2        3         Algeria
3        4  American Samoa
4        5         Andorra


In [11]:
# 7. DIMENSÃO RUNTIME

try:
    with conn.cursor() as cur:
        print("Criando dimensão Runtime...")
        
        sql_dim_runtime = """
        DROP TABLE IF EXISTS gold.dim_rte CASCADE;
        CREATE TABLE gold.dim_rte AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY runtime) AS srk_rte,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            runtime AS rte

        FROM (
            -- Seleciona runtime únicos
            SELECT DISTINCT runtime
            FROM silver.filmes
            WHERE runtime IS NOT NULL
        ) sub;

        ALTER TABLE gold.dim_rte ADD PRIMARY KEY (srk_rte);
        """
        cur.execute(sql_dim_runtime)
        print("gold.dim_rte criada com sucesso.")

        df_dim_runtime = pd.read_sql_query("SELECT * FROM gold.dim_rte;", conn)
        print(f"Número de registros em gold.dim_rte: {len(df_dim_runtime)}")
        print(f"Número de colunas em gold.dim_rte: {len(df_dim_runtime.columns)}")
        print(df_dim_runtime.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Runtime...
gold.dim_rte criada com sucesso.
Número de registros em gold.dim_rte: 772
Número de colunas em gold.dim_rte: 2
   srk_rte  rte
0        1    1
1        2    2
2        3    3
3        4    4
4        5    5


In [12]:
# 8. DIMENSÃO GENRES

try:
    with conn.cursor() as cur:
        print("Criando dimensão Genres...")
        
        sql_dim_genres = """
        DROP TABLE IF EXISTS gold.dim_gen CASCADE;
        CREATE TABLE gold.dim_gen AS
        SELECT
            -- Surrogate Key
            ROW_NUMBER() OVER (ORDER BY genre) AS srk_gen,
            
            -- Atributo de Negócio
            genre AS gen

        FROM (
            -- Explode os gêneros separados por vírgula
            SELECT DISTINCT
                TRIM(genre) AS genre
            FROM silver.filmes,
                 LATERAL unnest(string_to_array(genres, ',')) AS genre
            WHERE genres IS NOT NULL
        ) sub;

        ALTER TABLE gold.dim_gen ADD PRIMARY KEY (srk_gen);
        """
        
        cur.execute(sql_dim_genres)
        print("gold.dim_gen criada com sucesso.")

        df_dim_genres = pd.read_sql_query(
            "SELECT * FROM gold.dim_gen ORDER BY srk_gen;",
            conn
        )

        print(f"Número de registros em gold.dim_gen: {len(df_dim_genres)}")
        print(f"Número de colunas em gold.dim_gen: {len(df_dim_genres.columns)}")
        print(df_dim_genres.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Genres...
gold.dim_gen criada com sucesso.
Número de registros em gold.dim_gen: 19
Número de colunas em gold.dim_gen: 2
   srk_gen        gen
0        1     Action
1        2  Adventure
2        3  Animation
3        4     Comedy
4        5      Crime


In [13]:
# 9. DIMENSÃO PRODUCTION COMPANIES

try:
    with conn.cursor() as cur:
        print("Criando dimensão Production Companies...")
        
        sql_dim_production_companies = """
        DROP TABLE IF EXISTS gold.dim_prd_cmp CASCADE;
        CREATE TABLE gold.dim_prd_cmp AS
        SELECT
            -- Surrogate Key
            ROW_NUMBER() OVER (ORDER BY company) AS srk_prd,
            
            -- Atributo de Negócio
            company AS prd

        FROM (
            -- Explode as produtoras separadas por vírgula
            SELECT DISTINCT
                TRIM(company) AS company
            FROM silver.filmes,
                 LATERAL unnest(string_to_array(production_companies, ',')) AS company
            WHERE production_companies IS NOT NULL
        ) sub;

        ALTER TABLE gold.dim_prd_cmp ADD PRIMARY KEY (srk_prd);
        """
        
        cur.execute(sql_dim_production_companies)
        print("gold.dim_prd_cmp criada com sucesso.")

        df_dim_production_companies = pd.read_sql_query(
            "SELECT * FROM gold.dim_prd_cmp ORDER BY srk_prd;",
            conn
        )

        print(f"Número de registros em gold.dim_prd_cmp: {len(df_dim_production_companies)}")
        print(f"Número de colunas em gold.dim_prd_cmp: {len(df_dim_production_companies.columns)}")
        print(df_dim_production_companies.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Production Companies...
gold.dim_prd_cmp criada com sucesso.
Número de registros em gold.dim_prd_cmp: 164247
Número de colunas em gold.dim_prd_cmp: 2
   srk_prd  prd
0        1     
1        2  두리컴
2        3  딴생각
3        4  배선환
4        5  씨네넷


In [14]:
# 10. FATO FILME

try:
    with conn.cursor() as cur:
        print("Criando tabela fato filme...")
        
        # Etapa 1: Criar a Tabela com os Join
        sql_fact_movie = """
        DROP TABLE IF EXISTS gold.fat_mov CASCADE;

        CREATE TABLE gold.fat_mov AS
        SELECT
            -- Criação da Surrogate Key do Fato (SRK)
            ROW_NUMBER() OVER (ORDER BY f.id) AS srk_ttl,
            
            -- Atributos do Filme
            f.title                   AS ttl,
            f.original_title          AS org_ttl,
            f.created_at              AS crt,

            -- Chaves Estrangeiras para Dimensões
            dr.srk_rel,
            dp.srk_pft,
            de.srk_eng,
            dl.srk_lng,
            drt.srk_rte

        FROM silver.filmes f
        
        -- 1. Dimensão Release
        LEFT JOIN gold.dim_rel dr 
        ON f.release_date = dr.rel_dat

        -- 2. Dimensão Profit
        LEFT JOIN gold.dim_pft dp
        ON f.budget = dp.bdg AND f.revenue = dp.rev AND f.profit = dp.pft
        
        -- 3. Dimensão Engagement
        LEFT JOIN gold.dim_eng de
        ON f.popularity = de.pop AND f.vote_average = de.vot_avg AND f.vote_count = de.vot_cnt
        
        -- 4. Dimensão Language
        LEFT JOIN gold.dim_lng dl
        ON f.original_language = dl.lng

        -- 5. Dimensão Runtime
        LEFT JOIN gold.dim_rte drt
        ON f.runtime = drt.rte
        
        WHERE f.title IS NOT NULL;
       """
        cur.execute(sql_fact_movie)
        print("Tabela gold.fat_mov criada (dados inseridos).")

        # Adicionar Chaves Primárias e Estrangeiras
        print("Aplicando chaves e restrições...")
        sql_constraints = """
        ALTER TABLE gold.fat_mov ADD PRIMARY KEY (srk_ttl);

        -- Chave Estrangeira para Dimensões
        ALTER TABLE gold.fat_mov ADD CONSTRAINT frk_rel FOREIGN KEY (srk_rel) REFERENCES gold.dim_rel(srk_rel);
        ALTER TABLE gold.fat_mov ADD CONSTRAINT frk_pft FOREIGN KEY (srk_pft) REFERENCES gold.dim_pft(srk_pft);
        ALTER TABLE gold.fat_mov ADD CONSTRAINT frk_eng FOREIGN KEY (srk_eng) REFERENCES gold.dim_eng(srk_eng);
        ALTER TABLE gold.fat_mov ADD CONSTRAINT frk_lng FOREIGN KEY (srk_lng) REFERENCES gold.dim_lng(srk_lng);
        ALTER TABLE gold.fat_mov ADD CONSTRAINT frk_rte FOREIGN KEY (srk_rte) REFERENCES gold.dim_rte(srk_rte);

        """
        cur.execute(sql_constraints)
        print("Chaves aplicadas com sucesso.")

        print("Criando índices de performance...")
        sql_indexes = """
        CREATE INDEX idx_fat_mov_srk_rel ON gold.fat_mov(srk_rel);
        CREATE INDEX idx_fat_mov_srk_pft ON gold.fat_mov(srk_pft);
        CREATE INDEX idx_fat_mov_srk_eng ON gold.fat_mov(srk_eng);
        CREATE INDEX idx_fat_mov_srk_lng ON gold.fat_mov(srk_lng);
        CREATE INDEX idx_fat_mov_srk_rte ON gold.fat_mov(srk_rte);
        """
        cur.execute(sql_indexes)
        print("Índices criados com sucesso.")

        # Validação
        df_fact_movie = pd.read_sql_query("SELECT * FROM gold.fat_mov LIMIT 5;", conn)
        print(f"Colunas: {list(df_fact_movie.columns)}")
        print(df_fact_movie.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução da Fato: {e}")

finally:
    conn.commit()

Criando tabela fato filme...
Tabela gold.fat_mov criada (dados inseridos).
Aplicando chaves e restrições...
Chaves aplicadas com sucesso.
Criando índices de performance...
Índices criados com sucesso.
Colunas: ['srk_ttl', 'ttl', 'org_ttl', 'crt', 'srk_rel', 'srk_pft', 'srk_eng', 'srk_lng', 'srk_rte']
   srk_ttl                               ttl  \
0        1                             Ariel   
1        2               Shadows in Paradise   
2        3                        Four Rooms   
3        4                    Judgment Night   
4        5  Life in Loops (A Megacities RMX)   

                            org_ttl                        crt  srk_rel  \
0                             Ariel 2026-01-25 03:37:37.698907    24171   
1              Varjoja paratiisissa 2026-01-25 03:37:37.698907    23436   
2                        Four Rooms 2026-01-25 03:37:37.698907    26776   
3                    Judgment Night 2026-01-25 03:37:37.698907    25991   
4  Life in Loops (A Megacities RMX

In [41]:
# Criação da tabela ponte genre-filme na camada Gold
try:
    with conn.cursor() as cur:
        print("Criando tabela ponte genre-filme...")
        
        sql_fact_genre_movie = """
        DROP TABLE IF EXISTS gold.dim_gen_mov CASCADE;

        CREATE TABLE gold.dim_gen_mov AS
        WITH genres_exploded AS (
            SELECT
                fm.srk_ttl,
                TRIM(genre) AS genre,
                ROW_NUMBER() OVER (PARTITION BY sf.title ORDER BY sf.title) AS genre_position
            FROM silver.filmes sf
            JOIN gold.fat_mov fm ON sf.title = fm.ttl,
            LATERAL unnest(string_to_array(sf.genres, ',')) AS genre
            WHERE sf.genres IS NOT NULL
        ),
        genres_ranked AS (
            SELECT
                ge.srk_ttl,
                dg.srk_gen,
                (ge.genre_position = 1) AS pri_gen,
                ROW_NUMBER() OVER (PARTITION BY ge.srk_ttl, dg.srk_gen ORDER BY ge.genre_position) AS rn
            FROM genres_exploded ge
            JOIN gold.dim_gen dg ON TRIM(dg.gen) = ge.genre
        )
        SELECT
            srk_gen,
            srk_ttl,
            pri_gen
        FROM genres_ranked
        WHERE rn = 1;
        
        ALTER TABLE gold.dim_gen_mov ADD PRIMARY KEY (srk_ttl, srk_gen);

        ALTER TABLE gold.dim_gen_mov 
        ADD CONSTRAINT frk_gen FOREIGN KEY (srk_gen) REFERENCES gold.dim_gen(srk_gen);

        ALTER TABLE gold.dim_gen_mov 
        ADD CONSTRAINT frk_mov FOREIGN KEY (srk_ttl) REFERENCES gold.fat_mov(srk_ttl);
        """
        
        cur.execute(sql_fact_genre_movie)
        print("gold.dim_gen_mov criada com sucesso.")

        df_fact_genre_movie_count = pd.read_sql_query("SELECT COUNT(*) FROM gold.dim_gen_mov;", conn)
        print(f"Número de registros em gold.dim_gen_mov: {df_fact_genre_movie_count.iloc[0,0]}")
        df_fact_genre_movie = pd.read_sql_query("SELECT * FROM gold.dim_gen_mov WHERE pri_gen = true LIMIT 5;", conn)
        print(f"Número de colunas em gold.dim_gen_mov: {len(df_fact_genre_movie.columns)}")
        print(df_fact_genre_movie.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando tabela ponte genre-filme...
gold.dim_gen_mov criada com sucesso.
Número de registros em gold.dim_gen_mov: 1680267
Número de colunas em gold.dim_gen_mov: 3
   srk_gen  srk_ttl  pri_gen
0        4        3     True
1        6        5     True
2        2        7     True
3        3        8     True
4        4        9     True


In [42]:
# Criação da tabela ponte country-filme na camada Gold
try:
    with conn.cursor() as cur:
        print("Criando tabela ponte country-filme...")
        
        sql_fact_country_movie = """
        DROP TABLE IF EXISTS gold.dim_ctr_mov CASCADE;

        CREATE TABLE gold.dim_ctr_mov AS
        WITH countries_exploded AS (
            SELECT
                fm.srk_ttl,
                TRIM(country) AS country,
                ROW_NUMBER() OVER (PARTITION BY sf.title ORDER BY sf.title) AS country_position
            FROM silver.filmes sf
            JOIN gold.fat_mov fm ON sf.title = fm.ttl,
            LATERAL unnest(string_to_array(sf.production_countries, ',')) AS country
            WHERE sf.production_countries IS NOT NULL
        ),
        countries_ranked AS (
            SELECT
                ce.srk_ttl,
                dc.srk_ctr,
                (ce.country_position = 1) AS pri_ctr,
                ROW_NUMBER() OVER (PARTITION BY ce.srk_ttl, dc.srk_ctr ORDER BY ce.country_position) AS rn
            FROM countries_exploded ce
            JOIN gold.dim_ctr dc ON TRIM(dc.prd_ctr) = ce.country
        )
        SELECT
            srk_ctr,
            srk_ttl,
            pri_ctr
        FROM countries_ranked
        WHERE rn = 1;
        
        ALTER TABLE gold.dim_ctr_mov ADD PRIMARY KEY (srk_ttl, srk_ctr);

        ALTER TABLE gold.dim_ctr_mov 
        ADD CONSTRAINT frk_ctr FOREIGN KEY (srk_ctr) REFERENCES gold.dim_ctr(srk_ctr);

        ALTER TABLE gold.dim_ctr_mov 
        ADD CONSTRAINT frk_mov FOREIGN KEY (srk_ttl) REFERENCES gold.fat_mov(srk_ttl);
        """
        
        cur.execute(sql_fact_country_movie)
        print("gold.dim_ctr_mov criada com sucesso.")

        df_fact_country_movie_count = pd.read_sql_query("SELECT COUNT(*) FROM gold.dim_ctr_mov;", conn)
        print(f"Número total de registros em gold.dim_ctr_mov: {df_fact_country_movie_count.iloc[0,0]}")
        df_fact_country_movie = pd.read_sql_query("SELECT * FROM gold.dim_ctr_mov LIMIT 5;", conn)
        print(f"Número de colunas em gold.dim_ctr_mov: {len(df_fact_country_movie.columns)}")
        print(df_fact_country_movie.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando tabela ponte country-filme...
gold.dim_ctr_mov criada com sucesso.
Número total de registros em gold.dim_ctr_mov: 1315301
Número de colunas em gold.dim_ctr_mov: 3
   srk_ctr  srk_ttl  pri_ctr
0       10        1    False
1       40        1    False
2       63        1    False
3       76        1    False
4      183        1    False


In [43]:
# Criação da tabela ponte company-filme na camada Gold
try:
    with conn.cursor() as cur:
        print("Criando tabela ponte company-filme...")
        
        sql_fact_company_movie = """
        DROP TABLE IF EXISTS gold.dim_cmp_mov CASCADE;

        CREATE TABLE gold.dim_cmp_mov AS
        WITH companies_exploded AS (
            SELECT
                fm.srk_ttl,
                TRIM(company) AS company,
                ROW_NUMBER() OVER (PARTITION BY sf.title ORDER BY sf.title) AS company_position
            FROM silver.filmes sf
            JOIN gold.fat_mov fm ON sf.title = fm.ttl,
            LATERAL unnest(string_to_array(sf.production_companies, ',')) AS company
            WHERE sf.production_companies IS NOT NULL
        ),
        companies_ranked AS (
            SELECT
                ce.srk_ttl,
                dc.srk_cmp,
                (ce.company_position = 1) AS pri_cmp,
                ROW_NUMBER() OVER (PARTITION BY ce.srk_ttl, dc.srk_cmp ORDER BY ce.company_position) AS rn
            FROM companies_exploded ce
            JOIN gold.dim_cmp dc ON TRIM(dc.prd_cmp) = ce.company
        )
        SELECT
            srk_cmp,
            srk_ttl,
            pri_cmp
        FROM companies_ranked
        WHERE rn = 1;
        
        ALTER TABLE gold.dim_cmp_mov ADD PRIMARY KEY (srk_ttl, srk_cmp);

        ALTER TABLE gold.dim_cmp_mov 
        ADD CONSTRAINT frk_cmp FOREIGN KEY (srk_cmp) REFERENCES gold.dim_cmp(srk_cmp);

        ALTER TABLE gold.dim_cmp_mov 
        ADD CONSTRAINT frk_mov FOREIGN KEY (srk_ttl) REFERENCES gold.fat_mov(srk_ttl);
        """
        
        cur.execute(sql_fact_company_movie)
        print("gold.dim_cmp_mov criada com sucesso.")

        df_fact_company_movie_count = pd.read_sql_query("SELECT COUNT(*) FROM gold.dim_cmp_mov;", conn)
        print(f"Número de registros em gold.dim_cmp_mov: {df_fact_company_movie_count.iloc[0,0]}")
        df_fact_company_movie = pd.read_sql_query("SELECT * FROM gold.dim_cmp_mov LIMIT 5;", conn)
        print(f"Número de colunas em gold.dim_cmp_mov: {len(df_fact_company_movie.columns)}")
        print(df_fact_company_movie.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando tabela ponte company-filme...
gold.dim_cmp_mov criada com sucesso.
Número de registros em gold.dim_cmp_mov: 1673925
Número de colunas em gold.dim_cmp_mov: 3
   srk_cmp  srk_ttl  pri_cmp
0    27471        1    False
1    51581        1    False
2   155073        1    False
3   155073        2    False
4     2426        3    False


## Validação e Testes

In [44]:
try:
    with conn.cursor() as cur:
        antes = pd.read_sql_query("SELECT COUNT(*) FROM silver.filmes WHERE title IS NOT NULL;", conn).iloc[0,0]
        depois = pd.read_sql_query("SELECT COUNT(*) FROM gold.fat_mov;", conn).iloc[0,0]
        if antes == depois:
            print(f"Validação bem-sucedida: {antes} registros na silver.filmes e {depois} registros na gold.fat_mov.")
        else:
            print(f"Atenção: {antes} registros na silver.filmes, mas {depois} registros na gold.fat_mov.")
except Exception as e:
    print(f"Erro durante a validação: {e}")

Validação bem-sucedida: 1111340 registros na silver.filmes e 1111340 registros na gold.fat_mov.
